# Data cleaning

In [20]:
import os
import datetime
import string

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Data loading and inspecting

In [27]:
DATA_DIR = 'data'
RAW_FILE = 'raw_data_cro.xlsx' 
RAW_DATA_PATH = os.path.join(os.pardidf['e'] = df[col_list].sum(axis=1)r, DATA_DIR, RAW_FILE)

In [28]:
df = pd.read_excel(RAW_DATA_PATH)

In [29]:
df.shape

(200226, 38)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200226 entries, 0 to 200225
Data columns (total 38 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   ID_osoba                               200226 non-null  object 
 1   Datum_testiranja                       200226 non-null  object 
 2   Rezultat                               199782 non-null  float64
 3   Lokacija                               200226 non-null  object 
 4   Ustanova_koja_je_napravila_testiranja  200226 non-null  object 
 5   Hospitalizacija                        200226 non-null  int64  
 6   Respirator                             200226 non-null  int64  
 7   preminuli                              200226 non-null  int64  
 8   dob                                    200226 non-null  object 
 9   spol                                   200226 non-null  object 
 10  županija                               200226 non-null  

In [31]:
# check for NaNs
df.isnull().any()

ID_osoba                                 False
Datum_testiranja                         False
Rezultat                                  True
Lokacija                                 False
Ustanova_koja_je_napravila_testiranja    False
Hospitalizacija                          False
Respirator                               False
preminuli                                False
dob                                      False
spol                                     False
županija                                 False
I                                        False
sumcomorbidity                           False
A                                        False
B                                        False
C                                        False
D                                        False
E                                        False
F                                        False
G                                        False
H                                        False
J            

In [32]:
# only test results seem to be an issue,
# lets see how many NaNs are there
df['Rezultat'].isnull().sum()

444

## Creating a working dataset

In [73]:
# list all comorbidities A-Z
comorbidity_list = list(string.ascii_uppercase)

# drop existing sumcomorbidity column, create new and drop comorbidites
df_tidy = df.copy(deep=True)
df_tidy.drop(columns='sumcomorbidity', inplace=True)
df_tidy['sum_comorbidity'] = df_tidy[comorbidity_list].sum(axis=1)
df_tidy.drop(columns=comorbidity_list, inplace=True)
df_tidy

,ID_osoba,Datum_testiranja,Rezultat,Lokacija,Ustanova_koja_je_napravila_testiranja,Hospitalizacija,Respirator,preminuli,dob,spol,županija,sum_comorbidity
0,000D955C52,2020-07-04,0.0,RIJEKA,MEDICINSKI FAKULTET RIJEKA,0,0,0,50,1,PRIMORSKO-GORANSKA,7
1,000FE7B77A,2020-08-19,0.0,POŽEGA,O.B.POŽEGA,0,0,0,83,1,POŽEŠKO-SLAVONSKA,12
2,0022233DC0,2020-08-17,0.0,OSIJEK,ZJZ OSJEČKO-BARANJSKE ŽUPANIJE,0,0,0,42,1,OSJEČKO-BARANJSKA,9
3,0026A3B8BB,2020-06-02,0.0,KARLOVAC,KL.ZA INFEKTIVNE BOLESTI,0,0,0,89,2,KARLOVAČKA,7
4,00317D99E3,2020-06-30,0.0,OSIJEK,K.B.C.OSIJEK,0,0,0,17,2,OSJEČKO-BARANJSKA,6
...,...,...,...,...,...,...,...,...,...,...,...,...
200221,FFB0780383,2020-08-29,0.0,ZAGREB,ZJZ REPUBLIKE HRVATSKE,0,0,0,52,1,ŽUPANIJA GRAD ZAGREB,3
200222,FFB6289722,2020-08-20,0.0,VUKOVAR,K.B.C.OSIJEK,0,0,0,29,2,ŽUPANIJA VUKOVARSKO-SRIJEMSKA,7
200223,FFD598C2D0,2020-07-14,0.0,ZADAR,ZJZ ZDRAVSTVO ZADAR,0,0,0,24,1,ŽUPANIJA ZADARSKA,1
200224,FFDE31DCA8,2020-08-27,0.0,PULA,ZJZ ISTARSKE ŽUPANIJE,0,0,0,54,(null),(null),0


In [108]:
# replace '(null)' with np.Nan
df_tidy = df_tidy.replace('(null)', np.NaN)
df_tidy = df_tidy.replace('""(null)""', np.NaN)
# check for NaNs
df_tidy.isnull().sum()

ID_osoba                                     0
Datum_testiranja                             0
Rezultat                                   444
Lokacija                                     0
Ustanova_koja_je_napravila_testiranja        0
Hospitalizacija                              0
Respirator                                   0
preminuli                                    0
dob                                        134
spol                                      9774
županija                                 16451
sum_comorbidity                              0
dtype: int64

In [171]:
# df_tidy.ID_osoba
df_tidy.Datum_testiranja = df_tidy.Datum_testiranja.apply(pd.to_datetime)
df_tidy.Rezultat = df_tidy.Rezultat.replace({1.0: True, 0.0: False})
# df_tidy.Lokacija
# df_tidy.Ustanova_koja_je_napravila_testiranja
df_tidy.Hospitalizacija = df_tidy.Hospitalizacija.replace({1: True, 0: False})
df_tidy.Respirator = df_tidy.Respirator.replace({1: True, 0: False})
df_tidy.preminuli = df_tidy.Respirator.replace({1: True, 0: False})
df_tidy.dob = df_tidy.dob.apply(pd.to_numeric)
df_tidy.spol = df_tidy.spol.replace({1.: 'M', 2.: 'F'})
df_tidy.županija = df_tidy.županija.str.replace('ŽUPANIJA', '').str.replace(' ', '')
# df_tidy.sum_comorbidity

In [172]:
# translate column names and explicitly state data type
columns_dtypes_dict = {
    'id': str, 
    'age': float, 
    'sex': str, 
    'location': str, 
    'county': str,
    'testing_date': datetime.datetime, 
    'testing_result': bool, 
    'testing_institute': str,
    'hospitalization': bool, 
    'respirator': bool, 
    'deceased': bool,
    'total_comorbidities': int,
}
# create empty dataframe with predifined data types
columns_dtypes_list = [(k, v) for k, v in columns_dtypes_dict.items()]
dtypes = np.dtype(columns_dtypes_list)
empty_matrix = np.empty(0, dtype=dtypes)
df_w = pd.DataFrame(empty_matrix)
df_w

,id,age,sex,location,county,testing_date,testing_result,testing_institute,hospitalization,respirator,deceased,total_comorbidities


In [173]:
df_w.id = df_tidy.ID_osoba
df_w.age = df_tidy.dob
df_w.sex = df_tidy.spol
df_w.location = df_tidy.Lokacija
df_w.county = df_tidy.županija
df_w.testing_date = df_tidy.Datum_testiranja
df_w.testing_result = df_tidy.Rezultat
df_w.testing_institute = df_tidy.Ustanova_koja_je_napravila_testiranja
df_w.hospitalization = df_tidy.Hospitalizacija
df_w.respirator = df_tidy.Respirator
df_w.deceased = df_tidy.preminuli
df_w.total_comorbidities = df_tidy.sum_comorbidity

In [174]:
df_w

,id,age,sex,location,county,testing_date,testing_result,testing_institute,hospitalization,respirator,deceased,total_comorbidities
0,000D955C52,50.0,M,RIJEKA,PRIMORSKO-GORANSKA,2020-07-04,False,MEDICINSKI FAKULTET RIJEKA,False,False,False,7
1,000FE7B77A,83.0,M,POŽEGA,POŽEŠKO-SLAVONSKA,2020-08-19,False,O.B.POŽEGA,False,False,False,12
2,0022233DC0,42.0,M,OSIJEK,OSJEČKO-BARANJSKA,2020-08-17,False,ZJZ OSJEČKO-BARANJSKE ŽUPANIJE,False,False,False,9
3,0026A3B8BB,89.0,F,KARLOVAC,KARLOVAČKA,2020-06-02,False,KL.ZA INFEKTIVNE BOLESTI,False,False,False,7
4,00317D99E3,17.0,F,OSIJEK,OSJEČKO-BARANJSKA,2020-06-30,False,K.B.C.OSIJEK,False,False,False,6
...,...,...,...,...,...,...,...,...,...,...,...,...
200221,FFB0780383,52.0,M,ZAGREB,GRADZAGREB,2020-08-29,False,ZJZ REPUBLIKE HRVATSKE,False,False,False,3
200222,FFB6289722,29.0,F,VUKOVAR,VUKOVARSKO-SRIJEMSKA,2020-08-20,False,K.B.C.OSIJEK,False,False,False,7
200223,FFD598C2D0,24.0,M,ZADAR,ZADARSKA,2020-07-14,False,ZJZ ZDRAVSTVO ZADAR,False,False,False,1
200224,FFDE31DCA8,54.0,NaN,PULA,NaN,2020-08-27,False,ZJZ ISTARSKE ŽUPANIJE,False,False,False,0


In [ ]:
df_w